---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [15]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [16]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [17]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    

   
    df = pd.DataFrame(columns=('State','RegionName'))

    filepath = 'university_towns.txt'  
    with open(filepath) as fp:  
        line = fp.readline()
        line=line.strip()
        cnt = 1
        while line:
           z=re.search(r'edit',line,re.M|re.I)       
           if z:
                line_state=re.sub('\[edit\]','',line)        
                current_state=line_state
           else:                                             
                line=re.sub(r' \(.*','',line) # replace text after "(" included with ""
                df = df.append({'State': current_state,'RegionName':line}, ignore_index=True)    
           line = fp.readline()
           line= line.strip() 
           cnt += 1    
    return df

In [18]:
def get_recession_values():
    
    filepath = 'gdplev.xls'      
    gdp=pd.read_excel(filepath,header=None,skiprows=220,nrows=64)
    gdp=gdp.iloc[:,4:6]
    gdp.rename(columns = {4:'quarter',5:'GDP'}, inplace = True)

    
    size_df=len(gdp)
    recession=False
    
    for index,row in gdp.iterrows():
        if((size_df>=(index+1)) & (index>1)):
            
            first_quarter_value=gdp.iloc[index-2,1]
            second_quarter_value=gdp.iloc[index-1,1]
            third_quarter_value=gdp.iloc[index,1]
            
            first_quarter_name=gdp.iloc[index-2,0]
            second_quarter_name=gdp.iloc[index-1,0]
            third_quarter_name=gdp.iloc[index,0]
            
            if((not recession)):
                if (((second_quarter_value-first_quarter_value)<0)& 
                    ((third_quarter_value-second_quarter_value)<0)):
                    recession =True 
                    
                    recession_start_value=first_quarter_value
                    recession_start=first_quarter_name
                    recession_bottom=third_quarter_name
                    recession_bottom_value=third_quarter_value
                    
            else:        
                if (recession_bottom_value>first_quarter_value):
                    
                    recession_bottom_value=first_quarter_value
                    recession_bottom=first_quarter_name
                    
            
                if(((second_quarter_value-first_quarter_value)>0)& 
                ((third_quarter_value-second_quarter_value)>0)
                ):
                    recession =False                    
                    recession_end_value=third_quarter_value
                    recession_end=third_quarter_name
                    
    
    return (recession_start,recession_end,recession_bottom)



In [19]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recession_start,recession_end,recession_bottom=get_recession_values()
    return recession_start

In [20]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    recession_start,recession_end,recession_bottom=get_recession_values()            
    return recession_end

In [21]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_start,recession_end,recession_bottom=get_recession_values()
    return recession_bottom

In [30]:
def convert_housing_data_to_quarters():
    import math
    import pandas as pd
    import numpy as np
    filepath = 'City_Zhvi_AllHomes.csv'  
    City_Zhvi_AllHomes=pd.read_csv(filepath,header=0,nrows=10731)
    columns=City_Zhvi_AllHomes.columns
    columns_useful=[]
    columns_output=[]

    for i in range(len(columns)):
        if(i==1):
            columns_useful.append(columns[i])
        if(i==2):
            columns_useful.append(columns[i])
        if(i>50):
            columns_useful.append(columns[i])
            
    City_Zhvi_AllHomes=City_Zhvi_AllHomes[columns_useful]
    for k in range(len(columns_useful)):
        if(k>1):
            year=(columns_useful[k])[:4]
            month=(columns_useful[k])[5:]
            quarter=int(math.ceil((int(month)/3)))
            column_name=year+"q"+str(quarter)            
            if not {column_name}.issubset(columns_output):
                columns_output.append(column_name)
        else:
            columns_output.append(columns_useful[k])
    for k in range(len(columns_useful)):
        if(k>1):        
            if(((k-1)%3)==0): 
                print(str(k))
                year=(columns_useful[k])[:4]
                month=(columns_useful[k])[5:]
                quarter=int(math.ceil((int(month)/3)))
                column_name=year+"q"+str(quarter)            
                City_Zhvi_AllHomes[column_name]=City_Zhvi_AllHomes.apply(lambda x: ((x[columns_useful[k]]+x[columns_useful[k-1]]+x[columns_useful[k-2]])/(int(not np.isnan(x[columns_useful[k]])) + int(not np.isnan(x[columns_useful[k-1]])) +int(not np.isnan(x[columns_useful[k-2]])))) if((int(not np.isnan(x[columns_useful[k]])) + int(not np.isnan(x[columns_useful[k-1]])) +int(not np.isnan(x[columns_useful[k-2]])))>0) else np.nan,axis=1)
    #Last quarter we do in a different way because it has only two months
    City_Zhvi_AllHomes['2016q3']=City_Zhvi_AllHomes.apply(lambda x: ((x['2016-07']+x['2016-08'])/(int(not np.isnan(x['2016-07'])) + int(not np.isnan(x['2016-08'])) )) if((int(not np.isnan(x['2016-07'])) + int(not np.isnan(x['2016-08'])))>0) else np.nan,axis=1)
    df_output=City_Zhvi_AllHomes[columns_output] 
    df_output['State'] = df_output['State'].replace(states)
    df_output.set_index(["State","RegionName"],inplace=True)
    df_output.sort_index(inplace=True)
    
    return df_output

In [37]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    def get_quarter_before(year_quarter):
        year=year_quarter[:4]
        quarter=year_quarter[-1]
        if(quarter==1):
            year_quarter_before=str(year)+"q4"
        else:
            quarter_before=int(quarter)-1
            year_quarter_before=str(year)+"q"+str(quarter_before)
        return year_quarter_before



    df_housing_data = convert_housing_data_to_quarters()
    start_quarter=get_recession_start()
    bottom_quarter=get_recession_bottom()
    start_quarter_before=get_quarter_before(start_quarter)
    df_housing_data['ratio']=df_housing_data[bottom_quarter]-df_housing_data[start_quarter_before]

    df_list_university_towns=get_list_of_university_towns() 
    df_list_university_towns['university_town']=True
    df_housing_data=df_housing_data.reset_index()
    df_housing_data = pd.merge(df_housing_data,df_list_university_towns,how='outer',on=['State','RegionName'])
    df_housing_data['university_town'] = df_housing_data['university_town'].replace({np.NaN: False})
    df_univ_town_price_ratio = df_housing_data[df_housing_data['university_town'] == True]['ratio']
    df_non_univ_town_price_ratio = df_housing_data[df_housing_data['university_town'] == False]['ratio']

    st, p = ttest_ind(df_univ_town_price_ratio.dropna(), df_non_univ_town_price_ratio.dropna())


    # get different and better values
    different = False
    if p <0.01:
        different = True

    # determine which type of town is better
    better = ""
    if df_univ_town_price_ratio.mean() > df_non_univ_town_price_ratio.mean():
        better = "university town"
    else:
        better = "non-university town"

    return different, p, better